In [104]:
import pandas as pd
import os
from math import cos, asin, sqrt, pi

# Part 1: Data Processing

In [15]:
# Load the sampled data
df = pd.read_csv('sampled.csv')

/Users/scottqin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,12,20,21,27,30,35,36,42,45,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
# We will clean the sampled data year by year. There are columns with different names showing the same types of data.
# We need to merge these columns together and delete the irrelevant columns
# From the given sample table we know there are 8 useful columns.

In [73]:
pd.set_option('display.max_columns', None)

In [81]:
# create a new datafram called cleaned
cleaned = pd.DataFrame()
# create new columns
pickup_datetime = []
fare_amount = []
pickup_longitude = []
pickup_latitude = []
dropoff_longitude = []
dropoff_latitude = []
passenger_count = []

In [82]:
# clean the data from 2009
temp = df.iloc[0:14400]
pickup_datetime += list(temp['Trip_Pickup_DateTime'])
fare_amount += list(temp['Fare_Amt'])
pickup_longitude += list(temp['Start_Lon'])
pickup_latitude += list(temp['Start_Lat'])
dropoff_longitude += list(temp['End_Lon'])
dropoff_latitude += list(temp['End_Lat'])
passenger_count += list(temp['Passenger_Count'])

In [83]:
# clean the data from 2010
temp = df.iloc[14400:28800]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [84]:
# clean the data from 2011
temp = df.iloc[28800:43200]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [85]:
# clean the data from 2012
temp = df.iloc[43200:57600]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [86]:
# clean the data from 2013
temp = df.iloc[57600:72000]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [87]:
# clean the data from 2014
temp = df.iloc[72000:86400]
pickup_datetime += list(temp['pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])

In [88]:
# clean the data from 2015
temp = df.iloc[86400:]
pickup_datetime += list(temp['tpep_pickup_datetime'])
fare_amount += list(temp['fare_amount'])
pickup_longitude += list(temp['pickup_longitude'])
pickup_latitude += list(temp['pickup_latitude'])
dropoff_longitude += list(temp['dropoff_longitude'])
dropoff_latitude += list(temp['dropoff_latitude'])
passenger_count += list(temp['passenger_count'])


In [89]:
# add all the clean columns to the clean table
cleaned['pickup_datetime'] = pickup_datetime
cleaned['fare_amount'] = fare_amount
cleaned['pickup_longitude'] = pickup_longitude
cleaned['pickup_latitude'] = pickup_latitude
cleaned['dropoff_longitude']= dropoff_longitude
cleaned['dropoff_latitude'] = dropoff_latitude
cleaned['passenger_count'] = passenger_count

In [97]:
# exclude data outside of nyc
cleaned = cleaned.loc[(cleaned['pickup_longitude'] > -74.24233) & (cleaned['pickup_longitude'] < -73.717047)]
cleaned = cleaned.loc[(cleaned['dropoff_longitude'] > -74.24233) & (cleaned['dropoff_longitude'] < -73.717047)]
cleaned = cleaned.loc[(cleaned['pickup_latitude'] > 40.560445) & (cleaned['pickup_latitude'] < 40.908524)]
cleaned = cleaned.loc[(cleaned['dropoff_latitude'] > 40.560445) & (cleaned['dropoff_latitude'] < 40.908524)]

In [107]:
# define a function to calculate the distance between coordinates
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))

In [108]:
# testing the function
distance(40.751500, -73.984090, 40.770418, -73.864958)

10.251851855874952

In [109]:
# apply the function to table and add a new distance column
cleaned['distance'] = cleaned[['pickup_latitude','pickup_longitude','dropoff_latitude', 
                              'dropoff_longitude']].apply(lambda x: distance(*x), axis=1)

In [111]:
os.makedirs('Github/4501-Project', exist_ok=True)  
cleaned.to_csv('Github/4501-Project/cleaned.csv') 

# Part 2: Storing Data

# Part 3: Understanding Data

# Part 4: Visualizing Data